In [2]:
pip install pybliometrics

Note: you may need to restart the kernel to use updated packages.


In [3]:
# library configuration: https://pybliometrics.readthedocs.io/en/latest/configuration.html
# The first time init() is run there will be a prompt to input the key and instoken
# API Key: 4fec016525bc7cf8febea7c81db45976
# Instoken: 03a9e22c2f85af0fcdd7f9415998446b


import pybliometrics

pybliometrics.scopus.init()


In [4]:
# Line of code to check where the configuration file is saved

pybliometrics.scopus.utils.constants.CONFIG_FILE

WindowsPath('C:/Users/FSzeliga/.config/pybliometrics.cfg')

In [5]:
# Import pacakages

from pybliometrics.scopus import ScopusSearch
from pybliometrics.scopus import AuthorSearch
import pandas as pd
import json
from tqdm import tqdm

In [6]:
# set of parameters

min_year = 2013
max_year = 2023

#### Applied Geography (APG)

In [7]:
# Query set up and Scopus search

apg_query = f'EXACTSRCTITLE(Applied+AND+Geography) AND PUBYEAR > {min_year-1} AND PUBYEAR < {max_year+1}' 

s1 = ScopusSearch(apg_query, verbose=True, download=True, refresh=180) # Refresh=180 means that the results will be downloaded again after 180 days (to ensure up-to-date information)


In [8]:
# Checking length of results

s1.get_results_size()

2193

In [9]:
# Conversion to dataframe

apg_df = pd.DataFrame(s1.results)

In [10]:
# Unique Values

unique_doi = apg_df['doi'].unique()
print(f"The total number of unique DOI values is {len(unique_doi)}")

unique_journal = apg_df['publicationName'].unique()
print(f"The total number of unique journal titles {len(unique_journal)}")

The total number of unique DOI values is 2193
The total number of unique journal titles 3


In [11]:
# Print Journal Unique Values
print(unique_journal)

['Applied Geography' 'Papers in Applied Geography'
 'The Black Sea from Paleogeography to Modern Navigation: Applied Maritime Geography and Oceanography']


In [12]:
# Filter the DataFrame to keep only rows where 'journal_title' is 'Applied Geography'
apg_df = apg_df[apg_df['publicationName'] == 'Applied Geography']

#### Cartography and Geographic Information Science (CaGIS)

In [13]:
# Query set up and Scopus search

cagis_query = f'EXACTSRCTITLE(Cartography+AND+Geographic+AND+Information+AND+Science) AND PUBYEAR > {min_year-1} AND PUBYEAR < {max_year+1}' 

s2 = ScopusSearch(cagis_query, verbose=True, download=True, refresh=180) # Refresh=180 means that the results will be downloaded again after 180 days (to ensure up-to-date information)

In [14]:
# Checking length of results

s2.get_results_size()

448

In [15]:
# Conversion to dataframe

cagis_df = pd.DataFrame(s2.results)

In [16]:
# Unique Values

unique_doi = cagis_df['doi'].unique()
print(f"The total number of unique DOI values is {len(unique_doi)}")

unique_journal = cagis_df['publicationName'].unique()
print(f"The total number of unique journal titles {len(unique_journal)}")

The total number of unique DOI values is 448
The total number of unique journal titles 1


In [17]:
# Print Journal Unique Values
print(unique_journal)

['Cartography and Geographic Information Science']


#### GIScience and Remote Sensing (G&RS)

In [18]:
# Query set up and Scopus search

gandrs_query = f'EXACTSRCTITLE(GIScience+AND+Remote+AND+Sensing) AND PUBYEAR > {min_year-1} AND PUBYEAR < {max_year+1}' 

s3 = ScopusSearch(gandrs_query, verbose=True, download=True, refresh=180) # Refresh=180 means that the results will be downloaded again after 180 days (to ensure up-to-date information)

In [19]:
# Checking length of results

s3.get_results_size()

700

In [20]:
# Conversion to dataframe

gandrs_df = pd.DataFrame(s3.results)

In [21]:
# Unique Values

unique_doi = gandrs_df['doi'].unique()
print(f"The total number of unique DOI values is {len(unique_doi)}")

unique_journal = gandrs_df['publicationName'].unique()
print(f"The total number of unique journal titles {len(unique_journal)}")

The total number of unique DOI values is 700
The total number of unique journal titles 2


In [22]:
# Print Journal Unique Values
print(unique_journal)

['GIScience and Remote Sensing'
 'Remote Sensing and GIScience: Challenges and Future Directions']


In [23]:
# Filter the DataFrame to keep only rows where 'journal_title' is 'GIScience and Remote Sensing'
gandrs_df = gandrs_df[gandrs_df['publicationName'] == 'GIScience and Remote Sensing']

#### International Journal of Digital Earth (IJDE)

In [24]:
# Query set up and Scopus search

ijde_query = f'EXACTSRCTITLE(International+AND+Journal+AND+of+AND+Digital+AND+Earth) AND PUBYEAR > {min_year-1} AND PUBYEAR < {max_year+1}' 

s4 = ScopusSearch(ijde_query, verbose=True, refresh=180) # Refresh=180 means that the results will be downloaded again after 180 days (to ensure up-to-date information)

In [25]:
# Checking length of results

s4.get_results_size()

992

In [26]:
# Conversion to dataframe

ijde_df = pd.DataFrame(s4.results)

In [27]:
# Unique Values

unique_doi = ijde_df['doi'].unique()
print(f"The total number of unique DOI values is {len(unique_doi)}")

unique_journal = ijde_df['publicationName'].unique()
print(f"The total number of unique journal titles {len(unique_journal)}")

The total number of unique DOI values is 992
The total number of unique journal titles 1


In [28]:
# Print Journal Unique Values
print(unique_journal)

['International Journal of Digital Earth']


#### ISPRS International Journal of Geo-information (IJGI)

In [29]:
# Query set up and Scopus search

ijgi_query = f'EXACTSRCTITLE(ISPRS+AND+International+AND+Journal+AND+of+AND+Geo-information) AND PUBYEAR > {min_year-1} AND PUBYEAR < {max_year+1}' 

s5 = ScopusSearch(ijgi_query, verbose=True, refresh=180) # Refresh=180 means that the results will be downloaded again after 180 days (to ensure up-to-date information)

In [30]:
# Checking length of results

s5.get_results_size()

4690

In [31]:
# Conversion to dataframe

ijgi_df = pd.DataFrame(s5.results)

In [32]:
# Unique Values

unique_doi = ijgi_df['doi'].unique()
print(f"The total number of unique DOI values is {len(unique_doi)}")

unique_journal = ijgi_df['publicationName'].unique()
print(f"The total number of unique journal titles {len(unique_journal)}")

The total number of unique DOI values is 4690
The total number of unique journal titles 1


In [33]:
# Print Journal Unique Values
print(unique_journal)

['ISPRS International Journal of Geo-Information']


#### International Journal of Geographical Information Science (IJGIS)

In [34]:
# Query set up and Scopus search

ijgis_query = f'EXACTSRCTITLE(International+AND+Journal+AND+of+AND+Geographical+AND+Information+AND+Science) AND PUBYEAR > {min_year-1} AND PUBYEAR < {max_year+1}' 

s6 = ScopusSearch(ijgis_query, verbose=True, refresh=180) # Refresh=180 means that the results will be downloaded again after 180 days (to ensure up-to-date information)

In [35]:
# Checking length of results

s6.get_results_size()

1264

In [36]:
# Conversion to dataframe

ijgis_df = pd.DataFrame(s6.results)

In [37]:
# Unique Values

unique_doi = ijgis_df['doi'].unique()
print(f"The total number of unique DOI values is {len(unique_doi)}")

unique_journal = ijgis_df['publicationName'].unique()
print(f"The total number of unique journal titles {len(unique_journal)}")

The total number of unique DOI values is 1264
The total number of unique journal titles 1


In [38]:
# Print Journal Unique Values
print(unique_journal)

['International Journal of Geographical Information Science']


#### Journal of Spatial Science (JSS)

In [39]:
# Query set up and Scopus search

jss_query = f'EXACTSRCTITLE(Journal+AND+of+AND+Spatial+AND+Science) AND PUBYEAR > {min_year-1} AND PUBYEAR < {max_year+1}' 

s7 = ScopusSearch(jss_query, verbose=True, refresh=180) # Refresh=180 means that the results will be downloaded again after 180 days (to ensure up-to-date information)

In [40]:
# Checking length of results

s7.get_results_size()

493

In [41]:
# Conversion to dataframe

jss_df = pd.DataFrame(s7.results)

In [42]:
# Unique Values

unique_doi = jss_df['doi'].unique()
print(f"The total number of unique DOI values is {len(unique_doi)}")

unique_journal = jss_df['publicationName'].unique()
print(f"The total number of unique journal titles {len(unique_journal)}")

The total number of unique DOI values is 493
The total number of unique journal titles 2


In [43]:
# Print Journal Unique Values
print(unique_journal)

['Journal of Spatial Information Science' 'Journal of Spatial Science']


In [44]:
# Filter the DataFrame to keep only rows where 'journal_title' is 'Journal of Spatial Science'
jss_df = jss_df[jss_df['publicationName'] == 'Journal of Spatial Science']

#### ISPRS Journal of Photogrammetry and Remote Sensing (P&RS)

In [45]:
# Query set up and Scopus search

pandrs_query = f'EXACTSRCTITLE(ISPRS+AND+Journal+AND+of+AND+Photogrammetry+AND+Remote+AND+Sensing) AND PUBYEAR > {min_year-1} AND PUBYEAR < {max_year+1}' 

s8 = ScopusSearch(pandrs_query, verbose=True, refresh=180) # Refresh=180 means that the results will be downloaded again after 180 days (to ensure up-to-date information)

In [46]:
# Checking length of results

s8.get_results_size()

2481

In [47]:
# Conversion to dataframe

pandrs_df = pd.DataFrame(s8.results)

In [48]:
# Unique Values

unique_doi = pandrs_df['doi'].unique()
print(f"The total number of unique DOI values is {len(unique_doi)}")

unique_journal = pandrs_df['publicationName'].unique()
print(f"The total number of unique journal titles {len(unique_journal)}")

The total number of unique DOI values is 2481
The total number of unique journal titles 2


In [49]:
# Print Journal Unique Values
print(unique_journal)

['ISPRS Open Journal of Photogrammetry and Remote Sensing'
 'ISPRS Journal of Photogrammetry and Remote Sensing']


In [50]:
# Filter the DataFrame to keep only rows where 'journal_title' is 'ISPRS Journal of Photogrammetry and Remote Sensing'
pandrs_df = pandrs_df[pandrs_df['publicationName'] == 'ISPRS Journal of Photogrammetry and Remote Sensing']

#### Transactions in GIS (TGIS)

In [51]:
# Query set up and Scopus search

tgis_query = f'EXACTSRCTITLE(Transactions+AND+in+AND+GIS) AND PUBYEAR > {min_year-1} AND PUBYEAR < {max_year+1}' 

s9 = ScopusSearch(tgis_query, verbose=True, refresh=180) # Refresh=180 means that the results will be downloaded again after 180 days (to ensure up-to-date information)

In [52]:
# Checking length of results

s9.get_results_size()

962

In [53]:
# Conversion to dataframe

tgis_df = pd.DataFrame(s9.results)

In [54]:
# Unique Values

unique_doi = tgis_df['doi'].unique()
print(f"The total number of unique DOI values is {len(unique_doi)}")

unique_journal = tgis_df['publicationName'].unique()
print(f"The total number of unique journal titles {len(unique_journal)}")

The total number of unique DOI values is 962
The total number of unique journal titles 1


In [55]:
# Print Journal Unique Values
print(unique_journal)

['Transactions in GIS']


#### ACM Transactions on Spatial Algorithms and Systems (TSAS)

In [56]:
# Query set up and Scopus search

tsas_query = f'EXACTSRCTITLE(ACM+AND+Transactions+AND+on+AND+Spatial+AND+Algorithms+AND+Systems) AND PUBYEAR > {min_year-1} AND PUBYEAR < {max_year+1}' 

s10 = ScopusSearch(tsas_query, verbose=True, refresh=180) # Refresh=180 means that the results will be downloaded again after 180 days (to ensure up-to-date information)

In [57]:
# Checking length of results

s10.get_results_size()

190

In [58]:
# Conversion to dataframe

tsas_df = pd.DataFrame(s10.results)

In [59]:
# Unique Values

unique_doi = tsas_df['doi'].unique()
print(f"The total number of unique DOI values is {len(unique_doi)}")

unique_journal = tsas_df['publicationName'].unique()
print(f"The total number of unique journal titles {len(unique_journal)}")

The total number of unique DOI values is 190
The total number of unique journal titles 1


In [60]:
# Print Journal Unique Values
print(unique_journal)

['ACM Transactions on Spatial Algorithms and Systems']


#### Combination of journal dataframes and export to json

In [61]:
# Concatenation of journals dataframes
scopus_api_df = pd.concat([apg_df, cagis_df, gandrs_df, ijde_df, ijgi_df, ijgis_df, jss_df, pandrs_df, tgis_df, tsas_df], ignore_index=True)
drop_columns = ['eid', 'pii', 'pubmed_id', 'subtype', 'afid', 'affilname',
           'author_afids', 'source_id', 'eIssn', 'aggregationType','volume', 
           'issueIdentifier', 'article_number', 'pageRange', 'citedby_count', 'openaccess', 'freetoread', 
           'freetoreadLabel', 'fund_acr', 'fund_no', 'fund_sponsor']

scopus_api_df = scopus_api_df.drop(columns = drop_columns)

In [62]:
# Drop rows where there is no creator data
scopus_api_df = scopus_api_df.dropna(subset=['creator'])

# First Author ID column
scopus_api_df['creator_id'] = scopus_api_df['author_ids'].apply(lambda x: x.split(';')[0] if pd.notna(x) else x)


The view of the data below shows that the API extracted multiple values for the columns "affiliation_city" and "affiliation_country", which refer to the first author of the article. Following further research on this issue using the Elsevier Search API - Interactive Documentation (https://dev.elsevier.com/search.html#!/Scopus_Search/ScopusSearch), it was noted that this seems to be a fault in the Scopus_Search API, because when searching for the author's details using the Author_Search API instead, only one affiliation name, city and country is returned in most cases for each author. 

In [63]:
# Overview of data
scopus_api_df.head(5)

,doi,title,subtypeDescription,creator,affiliation_city,affiliation_country,author_count,author_names,author_ids,coverDate,coverDisplayDate,publicationName,issn,description,authkeywords,creator_id
0,10.1016/j.apgeog.2023.103145,Trapped in dilemma: Inverted N-shaped EKC evid...,Article,Huang J.,Beijing;Hong Kong,China;Hong Kong,6,"Huang, Jing;Zhang, Dong;Zhang, Zhengfeng;Kong,...",57382814800;57221284350;55774175000;5868965180...,2023-12-01,December 2023,Applied Geography,01436228,The environmental Kuznets relationship between...,Ecological land | Economic growth | Inverted N...,57382814800
1,10.1016/j.apgeog.2023.103136,Finding home: Participatory geospatial mapping...,Article,Al-haddad R.E.,Ukhiya;Tucson,Bangladesh;United States,2,"Al-haddad, Robin Elizabeth;Rakshit, Pradipto V...",57220131060;57776198000,2023-12-01,December 2023,Applied Geography,01436228,Despite decades of using community-based parti...,Geo-positioning | Geographic oppression | Part...,57220131060
2,10.1016/j.apgeog.2023.103122,Diffusion of the Internet-of-Things (IoT): A f...,Article,Jamme H.T.,Tempe,United States,2,"Jamme, Huê Tâm;Connor, Dylan S.",57203571610;55215499600,2023-12-01,December 2023,Applied Geography,01436228,Internet-of-Things (IoT) innovations are recon...,Community change | France | Internet of Things...,57203571610
3,10.1016/j.apgeog.2023.103142,A multi-level framework for assessing the spat...,Article,Dong J.,Beijing,China,5,"Dong, Junwu;Wang, Yanhui;Yang, Yang;Luo, Xiaoy...",57565840500;56066264400;57225057111;5733714420...,2023-12-01,December 2023,Applied Geography,01436228,Rapid urbanization is reducing the limited urb...,Accessibility | Public spaces | SDG 11.7.1 | S...,57565840500
4,10.1016/j.apgeog.2023.103140,Urbanization shifts freshwater service flows i...,Article,Shen W.,Guangzhou,China,2,"Shen, Wenting;Liu, Zhenhuan",58675798400;55585649300,2023-12-01,December 2023,Applied Geography,01436228,Freshwater ecosystem services supply and deman...,Freshwater ecosystem service flow | Social-eco...,58675798400


In [64]:
# Checking lenght of df

print("The total number of extracted articles is", len(scopus_api_df))

The total number of extracted articles is 13878


In [65]:
# Check for missing values in each column
print(scopus_api_df.isnull().sum())

doi                       0
title                     0
subtypeDescription        0
creator                   0
affiliation_city         56
affiliation_country      56
author_count              0
author_names              0
author_ids                0
coverDate                 0
coverDisplayDate          0
publicationName           0
issn                   4685
description             284
authkeywords           1245
creator_id                0
dtype: int64


The results below show the number of rows with multiple values in the columns "affiliation_city" and " affiliation_country". 

In [66]:
# Checking for rows with more than one value in the specified columns
columns_to_check = ['affiliation_city', 'affiliation_country']

# Create a dictionary to store the count of rows with multiple values for each column
multiple_values_count = {}

# Iterate through the columns and count rows with more than one value
for column in columns_to_check:
    multiple_values_count[column] = scopus_api_df[scopus_api_df[column].str.contains(';', na=False)].shape[0]

# Display the counts
multiple_values_count

{'affiliation_city': 9186, 'affiliation_country': 9186}

The code below extracts details of the authors using the Author_Search API, searching by the first author ID number. The code then generates a new dataframe "affiliation_df". This dataframe and the original dataframe can then be merged with a left joint and the author's details can be replaced accordingly. 

In [67]:
# note: the default weekly quota for AuthorSearch is 5,000. Elsevier support extended it to 15,000 for my API key. 
# Initialise a list to store the extracted information
affiliation_details = []

# Unique creator_id values
unique_creator_ids = scopus_api_df['creator_id'].dropna().unique()

# Iterate through each author ID in the DataFrame
for author_id in tqdm(unique_creator_ids, desc="Processing authors"):
    try:
        # Perform the AuthorSearch
        search = AuthorSearch(f'AU-ID({author_id})', download=True, refresh=180)
        
        # Extract the required information if the author is found
        if search.authors:
            author_info = search.authors[0]  # There should be one match only
            affiliation_details.append({
                'author_id': author_id,
                'city': author_info.city,
                'country': author_info.country
            })
        else:
            affiliation_details.append({
                'author_id': author_id,
                'city': np.nan,
                'country': np.nan
            })
    except Exception as e:
        print(f"An error occurred while processing author ID {author_id}: {e}")

# Convert the list to a DataFrame
affiliation_df = pd.DataFrame(affiliation_details)

# Rename ID column
affiliation_df.rename(columns={'author_id': 'creator_id'}, inplace=True)



Processing authors: 100%|██████████| 10441/10441 [00:03<00:00, 2900.57it/s]


In [68]:
affiliation_df.head(5)

,creator_id,city,country
0,57382814800,Beijing,China
1,57220131060,Tucson,United States
2,57203571610,Tempe,United States
3,57565840500,Beijing,China
4,58675798400,Guangzhou,China


Now there are no rows with multiple values in the columns "city" or "country".

In [69]:
# Checking for rows with more than one value in the specified columns
columns_to_check2 = ['city', 'country']

# Create a dictionary to store the count of rows with multiple values for each column
multiple_values_count2 = {}

# Iterate through the columns and count rows with more than one value
for column in columns_to_check2:
    multiple_values_count2[column] = affiliation_df[affiliation_df[column].str.contains(';', na=False)].shape[0]

# Display the counts
multiple_values_count2

{'city': 0, 'country': 0}

In [70]:
# Merge the two dataframes
merged_df = scopus_api_df.merge(affiliation_df, on='creator_id', how='left')

# Update of details based on merged_df
scopus_api_df['affiliation_city'] = merged_df['city']
scopus_api_df['affiliation_country'] = merged_df['country']

scopus_api_df.head(5)

,doi,title,subtypeDescription,creator,affiliation_city,affiliation_country,author_count,author_names,author_ids,coverDate,coverDisplayDate,publicationName,issn,description,authkeywords,creator_id
0,10.1016/j.apgeog.2023.103145,Trapped in dilemma: Inverted N-shaped EKC evid...,Article,Huang J.,Beijing,China,6,"Huang, Jing;Zhang, Dong;Zhang, Zhengfeng;Kong,...",57382814800;57221284350;55774175000;5868965180...,2023-12-01,December 2023,Applied Geography,01436228,The environmental Kuznets relationship between...,Ecological land | Economic growth | Inverted N...,57382814800
1,10.1016/j.apgeog.2023.103136,Finding home: Participatory geospatial mapping...,Article,Al-haddad R.E.,Tucson,United States,2,"Al-haddad, Robin Elizabeth;Rakshit, Pradipto V...",57220131060;57776198000,2023-12-01,December 2023,Applied Geography,01436228,Despite decades of using community-based parti...,Geo-positioning | Geographic oppression | Part...,57220131060
2,10.1016/j.apgeog.2023.103122,Diffusion of the Internet-of-Things (IoT): A f...,Article,Jamme H.T.,Tempe,United States,2,"Jamme, Huê Tâm;Connor, Dylan S.",57203571610;55215499600,2023-12-01,December 2023,Applied Geography,01436228,Internet-of-Things (IoT) innovations are recon...,Community change | France | Internet of Things...,57203571610
3,10.1016/j.apgeog.2023.103142,A multi-level framework for assessing the spat...,Article,Dong J.,Beijing,China,5,"Dong, Junwu;Wang, Yanhui;Yang, Yang;Luo, Xiaoy...",57565840500;56066264400;57225057111;5733714420...,2023-12-01,December 2023,Applied Geography,01436228,Rapid urbanization is reducing the limited urb...,Accessibility | Public spaces | SDG 11.7.1 | S...,57565840500
4,10.1016/j.apgeog.2023.103140,Urbanization shifts freshwater service flows i...,Article,Shen W.,Guangzhou,China,2,"Shen, Wenting;Liu, Zhenhuan",58675798400;55585649300,2023-12-01,December 2023,Applied Geography,01436228,Freshwater ecosystem services supply and deman...,Freshwater ecosystem service flow | Social-eco...,58675798400


In [71]:
# Checking the length of the dataframe to ensure that it has not changed
len(scopus_api_df)

13878

Once the dataframes are joined and the data is replaced, there are no rows with multiple affiliation countries and cities.

In [72]:
# Checking for rows with more than one value in the specified columns
columns_to_check = ['affiliation_city', 'affiliation_country']

# Create a dictionary to store the count of rows with multiple values for each column
multiple_values_count = {}

# Iterate through the columns and count rows with more than one value
for column in columns_to_check:
    multiple_values_count[column] = scopus_api_df[scopus_api_df[column].str.contains(';', na=False)].shape[0]

# Display the counts
multiple_values_count

{'affiliation_city': 0, 'affiliation_country': 0}

In [73]:
# Export df to json file
scopus_api_df.to_json('scopus_api_top10.json', orient='records', lines=True)

#### Read in json file

In [ ]:
# Read Scopus .json file
scopus_api_df = pd.read_json('scopus_api_top10.json', orient='records', lines=True)